### <b>Подготовка и загрузка текстовых документов</b>

In [1]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
from pathlib import Path

BASE_DIR: Path = Path().resolve().parent

Путь до каталога с документами

In [3]:
DOCUMENTS_DIR: Path = BASE_DIR / "documents"

Чтение и загрузка файлов

In [4]:
directory = Path(DOCUMENTS_DIR)
files = directory.iterdir()

texts = []
for file in files:
    with open(file=file, mode='r', encoding='utf-8') as f:
        text = f.read()
        print(f"---Count of chars: {len(text)}---")
        print(text)
        texts.append(text)
        
len(texts)

---Count of chars: 40227---
Памятка новичку
 
1.	Основная программа для оформления заданий и ЛУРВ (листы учета рабочего времени) 1С УФФ (управление фирмы-франчайзи). 
Инструкции по оформлению данных процессов размещены Документооборот - Документы и файлы – Файлы - Справочная информация.
2.	Вторая по значимости программа 1С Документооборот, с помощью нее мы оформляем следующие процессы: отпуск, день за свой счет, выход на работу в выходной день и т.д.
В этой программе размещена информация по сотрудникам (дни рождения, телефонный справочник). Документооборот - Документы и файлы - Файлы - Справочная информация.
3.	1С Коннект, внутренний сервис, используемый для общения с коллегами и клиентами.
4.	В компании также существует внутренний ресурс Public, где размещена информация по обучению в папке STUDY. В этой папке также создается папка с названием «Фамилия нового сотрудника», где вы можете размещать необходимые для вашей работы материалы.

Карьерная лестница: Серсив инженер -> младший спец

3

In [5]:
text = '\n\n'.join(texts)

print(f"---Общая длина текста: {len(text)}---")

---Общая длина текста: 67972---


### <b>Разделение текста на чанки</b>

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=20,
    length_function=len
)

chunks = text_splitter.create_documents([text])

N = 5

print(f"Всего чанков: {len(chunks)}")
print(f"Первые {N} чанков:")
chunks[:N]

Всего чанков: 154
Первые 5 чанков:


[Document(metadata={}, page_content='Памятка новичку\n \n1.\tОсновная программа для оформления заданий и ЛУРВ (листы учета рабочего времени) 1С УФФ (управление фирмы-франчайзи). \nИнструкции по оформлению данных процессов размещены Документооборот - Документы и файлы – Файлы - Справочная информация.\n2.\tВторая по значимости программа 1С Документооборот, с помощью нее мы оформляем следующие процессы: отпуск, день за свой счет, выход на работу в выходной день и т.д.\nВ этой программе размещена информация по сотрудникам (дни рождения, телефонный справочник). Документооборот - Документы и файлы - Файлы - Справочная информация.'),
 Document(metadata={}, page_content='3.\t1С Коннект, внутренний сервис, используемый для общения с коллегами и клиентами.\n4.\tВ компании также существует внутренний ресурс Public, где размещена информация по обучению в папке STUDY. В этой папке также создается папка с названием «Фамилия нового сотрудника», где вы можете размещать необходимые для вашей работы мат

### <b>Преобразование текста в векторное представление</b>

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

c:\Program Files\Python311\Lib\importlib\__init__.py:126: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  return _bootstrap._gcd_import(name[level:], package, level)


Загрузка embedding модели

In [ ]:
# intfloat/multilingual-e5-large

In [8]:
emdeddings = HuggingFaceEmbeddings(
    model_name="ai-forever/sbert_large_nlu_ru",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": False}
)

C:\Users\andre\AppData\Local\Temp\ipykernel_25552\2592369587.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emdeddings = HuggingFaceEmbeddings(


Создание локальной векторной базы данных (Chroma)

In [9]:
CHROMA_DIR = BASE_DIR / "chroma"

In [57]:
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_chroma import Chroma

In [13]:
vector_store = Chroma.from_documents(
    collection_name="dio-consult",
    documents=chunks,
    embedding=emdeddings,
    persist_directory=str(CHROMA_DIR)
)

: 